# WanGP v6.0 - Complete One-File Video Generation Platform

This notebook is a complete, self-contained solution that automatically:
- **Detects** your platform (Colab, Vast.ai, etc.).
- **Clones** or updates the required repository.
- **Installs** all dependencies.
- **Presents** a rich UI for model management and configuration.
- **Launches** the full WanGP interface.

**Just run all cells in order to get started!**

In [ ]:
# Cell 2: Environment Detection and Initial Setup
import os, sys, subprocess, platform, torch
from pathlib import Path
from IPython.display import display, HTML

# --- Enhanced Environment Detection ---
def detect_environment():
    env_markers = {
        'COLAB_GPU': 'colab', 'KAGGLE_KERNEL_RUN_TYPE': 'kaggle',
        'PAPERSPACE_GRADIENT_COMMIT': 'paperspace', 'VAST_CONTAINERLABEL': 'vast',
        'LIGHTNING_CLOUD_PROJECT_ID': 'lightning', 'RUNPOD_POD_ID': 'runpod'
    }
    for var, name in env_markers.items():
        if var in os.environ: return name
    if os.path.exists('/.dockerenv'): return 'docker'
    try:
        get_ipython(); return 'jupyter'
    except: return 'local'

ENVIRONMENT = detect_environment()
REPO_DIR = Path("wangp-standalone")
# ► IMPORTANT: Replace with your repository URL
REPO_URL = "https://github.com/YOUR_USERNAME/wangp-standalone.git"

print(f"🔍 Detected Environment: {ENVIRONMENT.upper()}")

# --- Clone or Update Repository ---
if not REPO_DIR.exists():
    print(f"📥 Cloning repository from {REPO_URL}...")
    subprocess.run(['git', 'clone', REPO_URL, str(REPO_DIR)], check=True)
else:
    print("🔄 Updating existing repository...")
    subprocess.run(['git', '-C', str(REPO_DIR), 'pull'], check=True)

os.chdir(REPO_DIR)
if str(Path.cwd() / 'src') not in sys.path:
    sys.path.insert(0, str(Path.cwd() / 'src'))
print(f"✅ Repository is ready in: {os.getcwd()}")

# --- System & GPU Information ---
SYSTEM_INFO = {
    'platform': platform.system(), 'python_version': sys.version.split()[0],
    'torch_version': torch.__version__, 'cuda_available': torch.cuda.is_available(),
    'cuda_version': torch.version.cuda if torch.cuda.is_available() else 'N/A'
}
if SYSTEM_INFO['cuda_available']:
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    SYSTEM_INFO['gpu_info_str'] = f"{gpu_name} ({gpu_mem:.1f} GB)"
else:
    SYSTEM_INFO['gpu_info_str'] = "Not Available"

display(HTML(f"<b>Platform:</b> {SYSTEM_INFO['platform']} | <b>Python:</b> {SYSTEM_INFO['python_version']} | <b>PyTorch:</b> {SYSTEM_INFO['torch_version']} | <b>CUDA:</b> {SYSTEM_INFO['cuda_version']} | <b>GPU:</b> {SYSTEM_INFO['gpu_info_str']}"))

In [ ]:
# Cell 3: Comprehensive Platform Setup
from src.platform_setup import PlatformSetup
import ipywidgets as widgets

progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Setup:', bar_style='info')
status_output = widgets.Output()
display(widgets.VBox([progress_bar, status_output]))

def update_progress(value, message):
    progress_bar.value = value
    progress_bar.description = f"{message[:15]}..."
    with status_output:
        print(message)

setup_manager = PlatformSetup(ENVIRONMENT, SYSTEM_INFO)
setup_manager.run(progress_callback=update_progress)

In [ ]:
# Cell 4: Interactive Model Selection and Download
from src.model_manager import ModelManager, ModelDownloader
from src.gpu_utils import GPUManager
import ipywidgets as widgets
from IPython.display import clear_output

# Initialize managers
model_manager = ModelManager()
gpu_manager = GPUManager()
gpu_info = gpu_manager.get_detailed_info()
available_models = model_manager.get_model_catalog()

# Create model selection widgets
model_checkboxes = {}
for model_id, model_info in available_models.items():
    is_compatible = gpu_info['vram_total_gb'] >= model_info.get('min_vram_gb', 6)
    checkbox = widgets.Checkbox(
        value=False,
        description=f"{model_info['name']} ({model_info.get('size', 'N/A')}) | VRAM: {model_info.get('min_vram_gb', 6)}GB",
        disabled=not is_compatible, indent=False, layout=widgets.Layout(width='95%')
    )
    model_checkboxes[model_id] = checkbox

select_recommended_button = widgets.Button(description='Select Recommended', button_style='info', icon='magic')
download_button = widgets.Button(description='Download Selected', button_style='primary', icon='download')
download_progress_output = widgets.Output()

def on_select_recommended(b):
    recommendations = gpu_manager.get_model_recommendations(gpu_info['vram_total_gb'])
    recommended_names = [model for models in recommendations.values() for model in models]
    for model_id, checkbox in model_checkboxes.items():
        if available_models[model_id]['name'] in recommended_names:
            checkbox.value = True

def on_download_models(b):
    selected_ids = [model_id for model_id, cb in model_checkboxes.items() if cb.value]
    if not selected_ids:
        with download_progress_output: clear_output(); print("⚠️ No models selected.")
        return

    downloader = ModelDownloader()
    with download_progress_output: clear_output(); print(f"📥 Starting download for {len(selected_ids)} models...")
    for model_id in selected_ids:
        with download_progress_output:
            print(f"   -> Downloading {available_models[model_id]['name']}...")
            downloader.download_model(model_id, available_models[model_id])
            print(f"   ✅ {available_models[model_id]['name']} ready.")

select_recommended_button.on_click(on_select_recommended)
download_button.on_click(on_download_models)

display(widgets.VBox([
    widgets.HTML("<h3>📦 Model Selection</h3>"),
    widgets.HBox([select_recommended_button, download_button]),
    widgets.VBox(list(model_checkboxes.values())),
    widgets.HTML("<h4>Download Status</h4>"),
    download_progress_output
]))

In [ ]:
# Cell 5: Configure Settings & Launch WanGP
from src.app import WanGPApp
from src.interface import create_gradio_interface
import threading, time

# Launch Controls
launch_button = widgets.Button(description="🚀 Launch WanGP!", button_style='success', layout=widgets.Layout(height='40px'))
stop_button = widgets.Button(description="⏹️ Stop Server", button_style='danger', layout=widgets.Layout(height='40px'), disabled=True)
status_display = widgets.Output()
url_display = widgets.HTML()

app_instance = None; app_thread = None

def on_launch_clicked(b):
    global app_instance, app_thread
    with status_display: clear_output(); print("🚀 Launching WanGP...")
    
    config = {
        'paths': {'model_cache_dir': os.environ['MODEL_CACHE'], 'lora_dir': os.environ.get('LORA_CACHE', './assets/loras'), 'output_dir': os.environ['OUTPUT_DIR']},
        'performance': {'precision': 'fp16', 'enable_xformers': True, 'cpu_offload': gpu_info['vram_total_gb'] < 12},
        'gpu_info': gpu_info
    }
    
    app_instance = WanGPApp(config)
    interface = create_gradio_interface(app_instance, config).queue()
    
    def run_app():
        interface.launch(share=True, prevent_thread_lock=True)
    
    app_thread = threading.Thread(target=run_app, daemon=True); app_thread.start()
    with status_display: print("🌐 Waiting for Gradio public URL... (this can take up to a minute)")
    time.sleep(15)
    
    url_display.value = """<div style='background-color: #e6ffed; padding: 15px; border-left: 5px solid #4caf50;'>
    <b>✅ WanGP is running!</b><br>
    Your public URL should have appeared in the output above. If not, check logs for errors.
    </div>"""
    launch_button.disabled = True; stop_button.disabled = False

def on_stop_clicked(b):
    # Kernel restart is the most reliable way to stop server threads and clear memory
    from IPython.core.display import Javascript
    display(Javascript('IPython.notebook.kernel.restart()'))

launch_button.on_click(on_launch_clicked)
stop_button.on_click(on_stop_clicked)

display(widgets.VBox([
    widgets.HTML("<h3>🎮 Launch Control</h3>"),
    widgets.HBox([launch_button, stop_button]),
    status_display, url_display
]))